In [1]:
import gzip
import io

import sys
import import_ipynb

from typing import TextIO, BinaryIO
from typing import List

sys.path.append('../')

from pre_processing import TextProcessor

importing Jupyter notebook from C:\Users\Davide\IR\Progetto\pre_processing\..\pre_processing\TextProcessor.ipynb


In [19]:
def parse_compressed_tsv_line_by_line(compressed_file_path):
    try:
        with open(compressed_file_path, 'rb') as f:
            text_processor = TextProcessor()
            with gzip.GzipFile(fileobj=f) as gz: 
                buffer = io.TextIOWrapper(gz, encoding='utf-8')

                # C'è bisogno di leggere la prima riga a parte perchè essa inizia con i seguenti metadati "# collection.tsv0000777000175000017502663675055413400073633015704 0ustar  spacemanidolspacemanidol0 "
                # Quindi prima di entrare nel ciclo che legge tutte le righe, elimino questo prefisso
                first_line = next(buffer)  # Leggi la prima riga
                cleaned_first_line = ' '.join(first_line.split()[3:])  # Rimuovi la parte iniziale indesiderata
                text_processor.process_text(text)(0, cleaned_first_line)

                for line in buffer:
                    pid, text = line.strip().split('\t')
                    text_processor.process_text(text)(pid, text)
                    print(pid,text)

    except FileNotFoundError:
        print(f"File {compressed_file_path} non trovato.")
    except Exception as e:
        print(e)
        print(f"Si è verificato un errore durante l'analisi del file {compressed_file_path}: {e}")

# Esempio di utilizzo
#parse_compressed_tsv_line_by_line('collection.tar.gz')


In [33]:
def decompress_strings(compressed_file_path):
    try:
        with open(compressed_file_path, 'rb') as f:
            text_processor = TextProcessor()
            with gzip.GzipFile(fileobj=f) as gz: 
                buffer = io.TextIOWrapper(gz, encoding='utf-8')
                print(buffer)
                # C'è bisogno di leggere la prima riga a parte perchè essa inizia con i seguenti metadati "# collection.tsv0000777000175000017502663675055413400073633015704 0ustar  spacemanidolspacemanidol0 "
                # Quindi prima di entrare nel ciclo che legge tutte le righe, elimino questo prefisso
                first_line = next(buffer)  # Leggi la prima riga
                cleaned_first_line = ' '.join(first_line.split()[3:])  # Rimuovi la parte iniziale indesiderata
                text_processor.process_text(text)(0, cleaned_first_line)

                for line in buffer:
                    pid, text = line.strip().split('\t')
                    text_processor.process_text(text)(pid, text)
                    #print(pid,text)

    except FileNotFoundError:
        print(f"File {compressed_file_path} non trovato.")
    except Exception as e:
        print(e)
        print(f"Si è verificato un errore durante l'analisi del file {compressed_file_path}: {e}")
    
    return decompressed_strings

In [ ]:
# Specifica il percorso del file TSV
file_path = "collection_cleaned.tsv"
with open(file_path, 'rb') as f:
    count = 0
    for line in f:
        print(line)
        count = count + 1
        if count == 5:
            break

In [20]:
parse_compressed_tsv_line_by_line("C:/Users/Davide/IR/collection.tar.gz")

'module' object is not callable
Si è verificato un errore durante l'analisi del file C:/Users/Davide/IR/collection.tar.gz: 'module' object is not callable


In [5]:
class Collection_Reader:
    
    collection_file:BinaryIO
        
    MEMORY_BASED_BUFFER_SIZE:int
    memory_based_buffer:List[str]
        
    text_processor:TextProcessor
    
    def __init__(self,path_collection_file:str,memory_based_buffer_size:int):
        
        print("Init Collection Reader")
        
        self.collection_file=open(path_collection_file, 'rb')
        self.MEMORY_BASED_BUFFER_SIZE=memory_based_buffer_size
        
        self.text_processor=TextProcessor()
        
        #Todo, qua aggiustarsi il caso che la collection abbia la prima riga composta da informazioni da scartare.
        

    def _read_collection(self):
        print("todo")
        #Todo
        #Leggere un numero di righe pari alla dimensione memory_based_buffer "in un colpo solo, quindi unica lettura"
        #eseguire quindi la decompressione dell'unico "blocco di righe letto"
        #e salvarle dentro memory_based_buffer
        
    
    
    def get_documents(self):
        """ This is the only function to be called from outside to have a list of documents ready to be processed.
        
        Returns:
           the memory_based_buffer list of strings
            
        """
        memory_based_buffer.clear()
        self.read_collection()
        return memory_based_buffer
    
    
    
    def close_file_collection(self):
        collection_file.close()
         
#Qua potrebbe essere utile pensare di farlo con 2 thread.
#Quando viene chiamata la funzione get_documents, si restituisce al chiamante il blocco di documenti richiesti
#e nel mentre si potrebbe attivare un thread che parallelamente carica il successivo blocco di documenti e lo
#salva localmente nella struttura dati, quindi alla prossima chiamata get_documents ho già pronto il blocco e non
# devo stare ad aspettare tutta la lettura.